Google Colab link : https://colab.research.google.com/drive/120IqNdb_44K1kSgqTTsZosf7VOaLO0ki?usp=sharing


# Preliminaries

In [ ]:
from google.colab import drive

drive.mount('/content/drive/')

Mounted at /content/drive/


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import LSTM, GRU, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

Dataset link : https://www.kaggle.com/datasets/tunguz/big-five-personality-test

In [ ]:
dataset = pd.read_csv("/content/drive/MyDrive/SEM4/Research Method/RM Kel 19 Experiment/data-final.csv", sep='\t')

print(dataset.shape)
dataset.head()

# Data Preprocessing

In [ ]:
data = dataset.drop(list(dataset)[50:], axis=1)

print(data.shape)
data.head()

(1015341, 50)


,EXT1,EXT2,EXT3,EXT4,EXT5,EXT6,EXT7,EXT8,EXT9,EXT10,...,OPN1,OPN2,OPN3,OPN4,OPN5,OPN6,OPN7,OPN8,OPN9,OPN10
0,4.0,1.0,5.0,2.0,5.0,1.0,5.0,2.0,4.0,1.0,...,5.0,1.0,4.0,1.0,4.0,1.0,5.0,3.0,4.0,5.0
1,3.0,5.0,3.0,4.0,3.0,3.0,2.0,5.0,1.0,5.0,...,1.0,2.0,4.0,2.0,3.0,1.0,4.0,2.0,5.0,3.0
2,2.0,3.0,4.0,4.0,3.0,2.0,1.0,3.0,2.0,5.0,...,5.0,1.0,2.0,1.0,4.0,2.0,5.0,3.0,4.0,4.0
3,2.0,2.0,2.0,3.0,4.0,2.0,2.0,4.0,1.0,4.0,...,4.0,2.0,5.0,2.0,3.0,1.0,4.0,4.0,3.0,3.0
4,3.0,3.0,3.0,3.0,5.0,3.0,3.0,5.0,3.0,4.0,...,5.0,1.0,5.0,1.0,5.0,1.0,5.0,3.0,5.0,5.0


In [ ]:
for i in data.columns:
  data = data[(data[i].notna()) & (data[i] != 0)]

print(data.shape)

(874434, 50)


In [ ]:
x = data.drop(columns=['EST9'])
y = data['EST9']

## Random oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
data['EST9'].value_counts()

In [ ]:
oversampler = RandomOverSampler(random_state=42)

In [ ]:
x_resampled, y_resampled = oversampler.fit_resample(x, y)

In [ ]:
y_resampled.value_counts()

3.0    247851
1.0    247851
4.0    247851
2.0    247851
5.0    247851
Name: EST9, dtype: int64

## XGBoost feature selection

In [ ]:
x_resampled = x_resampled[['EST6', 'EST8', 'AGR3', 'EST7', 'EST1', 'EST5', 'CSN9', 'EST10', 'AGR6', 'EST3']]

## Shaping dataset

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [ ]:
timesteps = 1
input_dim = 49

x_1 = np.resize(x, (x.shape[0], timesteps, input_dim))
x_train_1 = np.resize(x_train, (x_train.shape[0], timesteps, input_dim))
x_test_1 = np.resize(x_test, (x_test.shape[0], timesteps, input_dim))

In [ ]:
y_train_1 = y_train - 1
y_test_1 = y_test - 1

In [ ]:
y_train_1 = to_categorical(y_train_1)
y_test_1 = to_categorical(y_test_1)

print(y_test_1)

[[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 ...
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0.]]


# LSTM training

In [ ]:
x_train_1.shape

(786990, 1, 49)

In [ ]:
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(timesteps, input_dim)))
model.add(Dropout(0.5))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2460/2460 [==============================] - 27s 7ms/step - loss: 1.2531 - accuracy: 0.4574 - val_loss: 1.2083 - val_accuracy: 0.4801
Epoch 2/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.2177 - accuracy: 0.4781 - val_loss: 1.1841 - val_accuracy: 0.4962
Epoch 3/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.2045 - accuracy: 0.4860 - val_loss: 1.1765 - val_accuracy: 0.5004
Epoch 4/10
2460/2460 [==============================] - 16s 7ms/step - loss: 1.1977 - accuracy: 0.4897 - val_loss: 1.1673 - val_accuracy: 0.5040
Epoch 5/10
2460/2460 [==============================] - 17s 7ms/step - loss: 1.1940 - accuracy: 0.4927 - val_loss: 1.1707 - val_accuracy: 0.5076
Epoch 6/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.1898 - accuracy: 0.4953 - val_loss: 1.1621 - val_accuracy: 0.5089
Epoch 7/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.1869 - accuracy: 0.4964 - val_loss: 1.1559 - val_ac

In [ ]:
model.evaluate(x_test_1,y_test_1)

2733/2733 [==============================] - 9s 3ms/step - loss: 1.1519 - accuracy: 0.5129


[1.1518917083740234, 0.5128768086433411]

In [ ]:
import sklearn
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score

In [ ]:
y_pred_lstm = model.predict(x_test_1)
y_pred_lstm

2733/2733 [==============================] - 7s 2ms/step


array([[0.14376506, 0.35302565, 0.2334133 , 0.2246047 , 0.04519135],
       [0.0410495 , 0.4406325 , 0.31062928, 0.19262013, 0.01506868],
       [0.00420265, 0.04181146, 0.17969042, 0.6131048 , 0.16119064],
       ...,
       [0.272303  , 0.45826367, 0.18267886, 0.07699922, 0.00975527],
       [0.4574017 , 0.32994148, 0.11378101, 0.08011462, 0.01876132],
       [0.29034737, 0.33091387, 0.16857842, 0.16444144, 0.0457189 ]],
      dtype=float32)

In [ ]:
accuracy_score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1))

0.5128768125886282

In [ ]:
precision, recall, fscore, support = score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(np.argmax(y_test_1, axis=1), np.argmax(y_pred_lstm, axis=1)))

precision: [0.57152594 0.46276908 0.43656336 0.50538433 0.69284431]
recall: [0.59903554 0.55457967 0.3040271  0.60247897 0.49768431]
fscore: [0.58495749 0.50453165 0.35843587 0.5496769  0.57926829]
support: [11198 19925 18301 24849 13171]
################################
              precision    recall  f1-score   support

           0       0.57      0.60      0.58     11198
           1       0.46      0.55      0.50     19925
           2       0.44      0.30      0.36     18301
           3       0.51      0.60      0.55     24849
           4       0.69      0.50      0.58     13171

    accuracy                           0.51     87444
   macro avg       0.53      0.51      0.52     87444
weighted avg       0.52      0.51      0.51     87444



# GRU training

In [ ]:
modelGRU = Sequential()
modelGRU.add(GRU(128, return_sequences=True, input_shape=(timesteps, input_dim)))
modelGRU.add(Dropout(0.5))
modelGRU.add(GRU(64))
modelGRU.add(Dropout(0.5))
modelGRU.add(Dense(5, activation='softmax'))

In [ ]:
modelGRU.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelGRU.fit(x_train_1, y_train_1, epochs=10, batch_size=256, validation_split=0.2, callbacks=[EarlyStopping(monitor='val_loss', patience=7, min_delta=0.0001)])

Epoch 1/10
2460/2460 [==============================] - 19s 6ms/step - loss: 1.2559 - accuracy: 0.4545 - val_loss: 1.2022 - val_accuracy: 0.4825
Epoch 2/10
2460/2460 [==============================] - 15s 6ms/step - loss: 1.2175 - accuracy: 0.4784 - val_loss: 1.1824 - val_accuracy: 0.4936
Epoch 3/10
2460/2460 [==============================] - 14s 6ms/step - loss: 1.2037 - accuracy: 0.4875 - val_loss: 1.1766 - val_accuracy: 0.5024
Epoch 4/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.1951 - accuracy: 0.4919 - val_loss: 1.1633 - val_accuracy: 0.5067
Epoch 5/10
2460/2460 [==============================] - 15s 6ms/step - loss: 1.1911 - accuracy: 0.4939 - val_loss: 1.1633 - val_accuracy: 0.5102
Epoch 6/10
2460/2460 [==============================] - 16s 6ms/step - loss: 1.1879 - accuracy: 0.4971 - val_loss: 1.1567 - val_accuracy: 0.5131
Epoch 7/10
2460/2460 [==============================] - 15s 6ms/step - loss: 1.1855 - accuracy: 0.4985 - val_loss: 1.1611 - val_ac

In [ ]:
modelGRU.evaluate(x_test_1,y_test_1)

2733/2733 [==============================] - 9s 3ms/step - loss: 1.1541 - accuracy: 0.5131


[1.1541283130645752, 0.5131055116653442]

In [ ]:
y_pred_gru = modelGRU.predict(x_test_1)
y_pred_gru

2733/2733 [==============================] - 6s 2ms/step


array([[0.18099582, 0.31237036, 0.21597154, 0.22785175, 0.06281058],
       [0.04033691, 0.4025733 , 0.31611332, 0.22124636, 0.01973002],
       [0.00891864, 0.05591503, 0.15661131, 0.5469789 , 0.23157607],
       ...,
       [0.27708575, 0.48245645, 0.16222192, 0.06856579, 0.00967013],
       [0.38952866, 0.34466738, 0.13804354, 0.10265088, 0.02510963],
       [0.30249047, 0.26654583, 0.16528188, 0.18818969, 0.07749214]],
      dtype=float32)

In [ ]:
accuracy_score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1))

0.5131055303965967

In [ ]:
precision, recall, fscore, support = score(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1))

print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))
print('################################')
print(sklearn.metrics.classification_report(np.argmax(y_test_1, axis=1), np.argmax(y_pred_gru, axis=1)))

precision: [0.60801257 0.46953282 0.45396433 0.48191743 0.70362992]
recall: [0.55295588 0.55031368 0.24058795 0.68212    0.4827272 ]
fscore: [0.57917875 0.50672397 0.3145     0.56480232 0.57261224]
support: [11198 19925 18301 24849 13171]
################################
              precision    recall  f1-score   support

           0       0.61      0.55      0.58     11198
           1       0.47      0.55      0.51     19925
           2       0.45      0.24      0.31     18301
           3       0.48      0.68      0.56     24849
           4       0.70      0.48      0.57     13171

    accuracy                           0.51     87444
   macro avg       0.54      0.50      0.51     87444
weighted avg       0.52      0.51      0.50     87444

